In [1]:
import pyspark
from datetime import datetime
from pyspark.sql.types import IntegerType, FloatType
from pyspark.sql.functions import udf,lit

from pyspark.sql import SQLContext

sc = pyspark.SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

In [13]:
d = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true')\
        .load("BDdata/1994.csv")
for i in range(1995,2009):
    d = d.union(sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true')\
            .load('BDdata/'+str(i)+'.csv'))

In [3]:
d.show()

+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|1994|    1|         7|        5|    858|       900|    954|      1003|           US|      227

In [50]:
from pyspark.sql.functions import col

d2 = d.where(d1.TaxiIn != "NA").where(d1.TaxiOut != "NA").groupBy("Year","Origin","Dest").agg({"TaxiIn":"sum","TaxiOut":"sum"}).orderBy("Year","Origin","Dest")
dd2 = d2.groupBy("Year","Origin").agg({"sum(TaxiOut)":"sum"}).select("Year","Origin","sum(sum(TaxiOut))")
dd3 = d2.groupBy("Year","Dest").agg({"sum(TaxiIn)":"sum"}).select("Year","Dest","sum(sum(TaxiIn))")
#d2.coalesce(1).write.csv("./extraQuery/taxi",header=True)
dd4 = dd2.alias("a").join(dd3.alias("b"), on = [col("a.Origin") == col("b.Dest"), col("a.Year") == col("b.Year")])
dd4.show()

+----+------+-----------------+----+----+----------------+
|Year|Origin|sum(sum(TaxiOut))|Year|Dest|sum(sum(TaxiIn))|
+----+------+-----------------+----+----+----------------+
|1995|   FCA|           9369.0|1995| FCA|          4209.0|
|1998|   EWR|        3299816.0|1998| EWR|        995037.0|
|1998|   JNU|          50459.0|1998| JNU|         13645.0|
|1999|   FAT|           3586.0|1999| FAT|          1700.0|
|2000|   LIT|         129359.0|2000| LIT|         54305.0|
|2000|   MSP|        2779914.0|2000| MSP|        950838.0|
|2001|   MLU|          16237.0|2001| MLU|          7348.0|
|2002|   MBS|          20306.0|2002| MBS|          8328.0|
|2004|   ADQ|           3744.0|2004| ADQ|          1878.0|
|2004|   PHL|        2779238.0|2004| PHL|        919327.0|
|2005|   BZN|          63313.0|2005| BZN|         23264.0|
|2005|   CEC|           7894.0|2005| CEC|          2924.0|
|2005|   FSM|          19010.0|2005| FSM|         10219.0|
|2005|   PSE|           2238.0|2005| PSE|           781.

In [52]:
dd4.select("a.Year","Origin","sum(sum(TaxiOut))","sum(sum(TaxiIn))").withColumnRenamed("sum(sum(TaxiIn))","TaxiIn").withColumnRenamed("sum(sum(TaxiOut))","TaxiOut")\
    .orderBy("Year","Origin").coalesce(1).write.csv("./extraQuery/taxi",header=True)

In [38]:
d3 = d.groupBy("Year","Origin","Dest").agg({"*":"count"}).orderBy("Year","Origin","Dest")
d3.coalesce(1).write.csv("./extraQuery/from_to",header=True)